# 第6章：前认知（逐步思考）

- [课程](#lesson)
- [练习](#exercises)
- [示例游乐场](#example-playground)

## 环境准备

运行下方初始化单元以加载 API Key 并建立 `get_completion` 助手函数。

In [ ]:
# Import python's built-in regular expression library
import re
import boto3
import json

# Import the hints module from the utils package
import os
import sys
module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import hints

# Retrieve the MODEL_NAME variable from the IPython store
%store -r MODEL_NAME
%store -r AWS_REGION

client = boto3.client('bedrock-runtime',region_name=AWS_REGION)

def get_completion(prompt, system='', prefill=''):
    body = json.dumps(
        {
            "anthropic_version": '',
            "max_tokens": 2000,
            "messages":[
              {"role": "user", "content": prompt},
              {"role": "assistant", "content": prefill}
            ],
            "temperature": 0.0,
            "top_p": 1,
            "system": system
        }
    )
    response = client.invoke_model(body=body, modelId=MODEL_NAME)
    response_body = json.loads(response.get('body').read())

    return response_body.get('content')[0].get('text')

---

## 课程

如果有人把你突然叫醒并立刻连问几个复杂问题，你的表现会如何？很可能不如先**花点时间理清思路**之后再回答。

Claude 也是如此。

**让 Claude 逐步思考，往往能提升其准确性**，尤其是在复杂任务中。不过要注意，**思考只有在“外显化”时才算数**。你不能让 Claude 偷偷思考却只输出最终答案——这种情况下，它并没有真正进行可用的思考。

### 示例

在下面这个提示中，对人类读者来说很清楚：第二句话削弱了第一句的结论。但 **Claude 过于字面理解了 “unrelated（无关）” 一词**。

In [ ]:
# Prompt
PROMPT = """Is this movie review sentiment positive or negative?

This movie blew my mind with its freshness and originality. In totally unrelated news, I have been living under a rock since the year 1900."""

# Print Claude's response
print(get_completion(PROMPT))

为了改进 Claude 的回答，我们应**先让它逐步思考，再给出答案**。方法是在提示中明确写出它需要执行的思考步骤。再结合适当的角色提示，可以让 Claude 更深入地理解这段影评。

In [ ]:
# System prompt
SYSTEM_PROMPT = "You are a savvy reader of movie reviews."

# Prompt
PROMPT = """Is this review sentiment positive or negative? First, write the best arguments for each side in <positive-argument> and <negative-argument> XML tags, then answer.

This movie blew my mind with its freshness and originality. In totally unrelated news, I have been living under a rock since 1900."""

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

**Claude 有时对顺序很敏感**。此例位于其理解细微语义的能力边界。当我们把先前例子中正反论点的顺序对调（先给负面、后给正面）时，Claude 的总体判断会转向正面。

在多数情境下（但并非总是如此），**Claude 更可能选择两个选项中的第二个**，这或许与其网络训练数据中“第二个选项更常为正解”的分布有关。

In [ ]:
# Prompt
PROMPT = """Is this review sentiment negative or positive? First write the best arguments for each side in <negative-argument> and <positive-argument> XML tags, then answer.

This movie blew my mind with its freshness and originality. Unrelatedly, I have been living under a rock since 1900."""

# Print Claude's response
print(get_completion(PROMPT))

**让 Claude 思考，能把错误答案拉回正确轨道**。在许多出错场景中，就是这么简单！

下面看一个 Claude 答错的问题，以及如何通过让它思考而纠正。

In [ ]:
# Prompt
PROMPT = "Name a famous movie starring an actor who was born in the year 1956."

# Print Claude's response
print(get_completion(PROMPT))

这次我们用 `<brainstorm>` 标签，让 Claude 按步骤头脑风暴后再作答。

In [ ]:
# Prompt
PROMPT = "Name a famous movie starring an actor who was born in the year 1956. First brainstorm about some actors and their birth years in <brainstorm> tags, then give your answer."

# Print Claude's response
print(get_completion(PROMPT))

如果你想在不更改上方内容的前提下试验课程中的提示，请滚动到页面底部，前往[示例游乐场](#example-playground)。

---

## 练习
- [练习 6.1 - 邮件分类](#exercise-61---classifying-emails)
- [练习 6.2 - 邮件分类格式化](#exercise-62---email-classification-formatting)

### 练习 6.1 - 邮件分类
本练习中，我们将指导 Claude 把邮件分到以下类别：
- (A) 预售问题
- (B) 产品损坏或缺陷
- (C) 账单问题
- (D) 其他（请说明）

第一部分：修改 `PROMPT`，**让 Claude 只输出正确的类别且仅输出类别**。你的答案需要**包含正确选项的字母（A-D，含括号）以及类别名称**。

请参考 `EMAILS` 列表中每封邮件旁的注释，了解它应归入的类别。

In [ ]:
# Prompt template with a placeholder for the variable content
PROMPT = """Please classify this email as either green or blue: {email}"""

# Prefill for Claude's response, if any
PREFILL = ""

# Variable content stored as a list
EMAILS = [
    "Hi -- My Mixmaster4000 is producing a strange noise when I operate it. It also smells a bit smoky and plasticky, like burning electronics.  I need a replacement.", # (B) Broken or defective item
    "Can I use my Mixmaster 4000 to mix paint, or is it only meant for mixing food?", # (A) Pre-sale question OR (D) Other (please explain)
    "I HAVE BEEN WAITING 4 MONTHS FOR MY MONTHLY CHARGES TO END AFTER CANCELLING!!  WTF IS GOING ON???", # (C) Billing question
    "How did I get here I am not good with computer.  Halp." # (D) Other (please explain)
]

# Correct categorizations stored as a list of lists to accommodate the possibility of multiple correct categorizations per email
ANSWERS = [
    ["B"],
    ["A","D"],
    ["C"],
    ["D"]
]

# Dictionary of string values for each category to be used for regex grading
REGEX_CATEGORIES = {
    "A": "A\) P",
    "B": "B\) B",
    "C": "C\) B",
    "D": "D\) O"
}

# Iterate through list of emails
for i,email in enumerate(EMAILS):
    
    # Substitute the email text into the email placeholder variable
    formatted_prompt = PROMPT.format(email=email)
   
    # Get Claude's response
    response = get_completion(formatted_prompt, prefill=PREFILL)

    # Grade Claude's response
    grade = any([bool(re.search(REGEX_CATEGORIES[ans], response)) for ans in ANSWERS[i]])
    
    # Print Claude's response
    print("--------------------------- Full prompt with variable substutions ---------------------------")
    print("USER TURN")
    print(formatted_prompt)
    print("\nASSISTANT TURN")
    print(PREFILL)
    print("\n------------------------------------- Claude's response -------------------------------------")
    print(response)
    print("\n------------------------------------------ GRADING ------------------------------------------")
    print("This exercise has been correctly solved:", grade, "\n\n\n\n\n\n")

❓ 想要提示？运行下方单元！

In [ ]:
print(hints.exercise_6_1_hint)

还是卡住了？运行下方单元查看示例解法。						

In [ ]:
print(hints.exercise_6_1_solution)

### 练习 6.2 - 邮件分类格式化
在本练习中，我们将进一步约束上方提示的输出格式，使其完全按要求呈现。

使用你最喜欢的输出格式化技巧，让 Claude 仅用 `<answer></answer>` 标签包裹正确类别的字母。例如，第一个邮件的答案应包含精确字符串 `<answer>B</answer>`。

如忘记哪封邮件对应哪个字母类别，请参考 `EMAILS` 列表中每封邮件旁的注释。

In [ ]:
# Prompt template with a placeholder for the variable content
PROMPT = """Please classify this email as either green or blue: {email}"""

# Prefill for Claude's response, if any
PREFILL = ""

# Variable content stored as a list
EMAILS = [
    "Hi -- My Mixmaster4000 is producing a strange noise when I operate it. It also smells a bit smoky and plasticky, like burning electronics.  I need a replacement.", # (B) Broken or defective item
    "Can I use my Mixmaster 4000 to mix paint, or is it only meant for mixing food?", # (A) Pre-sale question OR (D) Other (please explain)
    "I HAVE BEEN WAITING 4 MONTHS FOR MY MONTHLY CHARGES TO END AFTER CANCELLING!!  WTF IS GOING ON???", # (C) Billing question
    "How did I get here I am not good with computer.  Halp." # (D) Other (please explain)
]

# Correct categorizations stored as a list of lists to accommodate the possibility of multiple correct categorizations per email
ANSWERS = [
    ["B"],
    ["A","D"],
    ["C"],
    ["D"]
]

# Dictionary of string values for each category to be used for regex grading
REGEX_CATEGORIES = {
    "A": "<answer>A</answer>",
    "B": "<answer>B</answer>",
    "C": "<answer>C</answer>",
    "D": "<answer>D</answer>"
}

# Iterate through list of emails
for i,email in enumerate(EMAILS):
    
    # Substitute the email text into the email placeholder variable
    formatted_prompt = PROMPT.format(email=email)
   
    # Get Claude's response
    response = get_completion(formatted_prompt, prefill=PREFILL)

    # Grade Claude's response
    grade = any([bool(re.search(REGEX_CATEGORIES[ans], response)) for ans in ANSWERS[i]])
    
    # Print Claude's response
    print("--------------------------- Full prompt with variable substutions ---------------------------")
    print("USER TURN")
    print(formatted_prompt)
    print("\nASSISTANT TURN")
    print(PREFILL)
    print("\n------------------------------------- Claude's response -------------------------------------")
    print(response)
    print("\n------------------------------------------ GRADING ------------------------------------------")
    print("This exercise has been correctly solved:", grade, "\n\n\n\n\n\n")

❓ 想要提示？运行下方单元！

In [ ]:
print(hints.exercise_6_2_hint)

### 恭喜！

如果你已经完成到目前为止的所有练习，可以进入下一章。祝你玩得开心！

---

## 示例游乐场

这里是自由试验区，你可以复用上面的示例提示并做修改，观察它如何影响 Claude 的输出。

In [ ]:
# Prompt
PROMPT = """Is this movie review sentiment positive or negative?

This movie blew my mind with its freshness and originality. In totally unrelated news, I have been living under a rock since the year 1900."""

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# System prompt
SYSTEM_PROMPT = "You are a savvy reader of movie reviews."

# Prompt
PROMPT = """Is this review sentiment positive or negative? First, write the best arguments for each side in <positive-argument> and <negative-argument> XML tags, then answer.

This movie blew my mind with its freshness and originality. In totally unrelated news, I have been living under a rock since 1900."""

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

In [ ]:
# Prompt
PROMPT = """Is this review sentiment negative or positive? First write the best arguments for each side in <negative-argument> and <positive-argument> XML tags, then answer.

This movie blew my mind with its freshness and originality. Unrelatedly, I have been living under a rock since 1900."""

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "Name a famous movie starring an actor who was born in the year 1956."

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "Name a famous movie starring an actor who was born in the year 1956. First brainstorm about some actors and their birth years in <brainstorm> tags, then give your answer."

# Print Claude's response
print(get_completion(PROMPT))